<a href="https://colab.research.google.com/github/gleidsonnunes/scripts/blob/master/2D_to_3D_SBS_Converter_with_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Conversor 2D para 3D SBS com MiDaS e Gradio

Este notebook usa IA (modelo MiDaS da Meta) para converter vídeos 2D em vídeos 3D Side-by-Side (SBS).

In [ ]:
#@title ⚙️  Install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -u -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.10/site-packages') # Adjust Python version if needed

In [ ]:
#@title ⚙️ Instalar dependências
!apt -y install ffmpeg
!pip install gradio opencv-python matplotlib torch torchvision torchaudio --quiet
!git clone https://github.com/isl-org/MiDaS.git
%cd MiDaS
!conda env create -f environment.yaml
!conda init --all
!conda activate midas-py310

In [ ]:
#@title 🎞️ Processa
import torch
import cv2
import numpy as np
import shutil
import gradio as gr
import uuid
import os
from pathlib import Path

# Dispositivo (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregar modelo MiDaS e transformações
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Large")
midas.to(device).eval()

midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

def generate_3d_image(image, depth_map, shift=8):
    depth = cv2.normalize(depth_map, None, 0, 1, cv2.NORM_MINMAX)
    depth = cv2.GaussianBlur(depth, (5, 5), 0)

    h, w = depth.shape
    left = np.zeros_like(image)
    right = np.zeros_like(image)

    for y in range(h):
        for x in range(w):
            offset = int((depth[y, x] - 0.5) * 2 * shift)
            if 0 <= x - offset < w:
                left[y, x] = image[y, x - offset]
            if 0 <= x + offset < w:
                right[y, x] = image[y, x + offset]

    return np.hstack((left, right))

def convert_video_to_3d_sbs(video_path):
    tmp_id = str(uuid.uuid4())
    os.makedirs("frames", exist_ok=True)
    os.makedirs("sbs_frames", exist_ok=True)
    os.makedirs("output", exist_ok=True)

    os.system(f"ffmpeg -i {video_path} -qscale:v 2 frames/frame_%05d.png")

    for filename in sorted(os.listdir("frames")):
      frame = cv2.imread(f"frames/{filename}")
      if frame is None:
          print(f"Error loading frame: frames/{filename}")
          continue

      img_input = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # Ensure image is in the correct format (H, W, C) before transforming
      if img_input.ndim == 3:
          input_tensor = transform(img_input).to(device)
          print(f"Shape of input_tensor before model: {input_tensor.shape}") # Add print for debugging
      else:
          print(f"Skipping frame {filename} due to incorrect dimensions: {img_input.shape}")
          continue


      with torch.no_grad():
          prediction = midas(input_tensor)
          prediction = torch.nn.functional.interpolate(
              prediction.unsqueeze(1),
              size=frame.shape[:2],
              mode="bicubic",
              align_corners=False,
          ).squeeze()

      depth_map = prediction.cpu().numpy()
      sbs = generate_3d_image(frame, depth_map, shift=8)
      cv2.imwrite(f"sbs_frames/{filename}", sbs)

    output_file = f"output/{tmp_id}_3d_sbs.mp4"
    os.system(f"ffmpeg -framerate 24 -i sbs_frames/frame_%05d.png -c:v libx264 -pix_fmt yuv420p {output_file}")

    shutil.rmtree("frames")
    shutil.rmtree("sbs_frames")

    return output_file

def process_interface(video, progress=gr.Progress(track_tqdm=True)):
    output_path = convert_video_to_3d_sbs(video)
    return output_path

gr.Interface(
    fn=process_interface,
    inputs=gr.Video(label="Vídeo 2D (.mp4)"),
    outputs=gr.Video(label="Resultado 3D SBS"),
    title="Conversor 2D para 3D com MiDaS",
    description="Envie um vídeo curto 2D para converter em 3D SBS com IA. Ideal para testes com VR."
).launch(share=True, debug=True)